In [1]:
import os, time
from pytorch_transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, CONFIG_NAME 
from pytorch_transformers import AdamW
from fp16 import FP16_Module, FP16_Optimizer
import json, csv
from multiprocessing import Pool
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from collections import OrderedDict
import logging
import warnings
import copy
from metrics import compute_metrics

In [2]:
data_dir = "/root/LAMOL/lamol_data"
MODEL_BASE_DIR = "/data/model_runs"
# MODEL_DIR_NAME = "20210630T184801_mbs_SEQ_MAML"
# MODEL_DIR_NAME = "20210701T033914_mbs_SEQ"
MODEL_DIR_NAME = "20210701T073948_msb_SEQ_MAML"
# MODEL_DIR_NAME = "20210702T034545_msb_SEQ"
# MODEL_DIR_NAME = "20210701T180911_sbm_SEQ_MAML"

MODEL_DIR_NAME = "20210824T030642_mbs_SEQ_MAML" # Special of movie_steps101.model. Save special in between
MODEL_DIR_NAME = "20210824T035319_mbs_SEQ_MAML_savenetpibetween" # special of movie_steps101.model 
MODEL_DIR_NAME = "20210824T053734_mbs_SEQ_MAML_loadstatedict" # special load state dict
MODEL_DIR_NAME = "20210829T003703_mbs_SEQ_MAML_v2" # special load state dict
MODEL_DIR_NAME = "20210905T094223_sbm_SEQ_MAML"

MODEL_DIR = os.path.join(MODEL_BASE_DIR,MODEL_DIR_NAME)

# tasks = ['movie',  'scifact', 'boolq']
# tasks = ['movie', 'boolq',  'scifact']
tasks = [ 'scifact', 'boolq', 'movie']

DEVICE = 'cuda:0'
temperature_qa = 1.0
n_train_epochs = 1
num_updates = 5

train_batch_size = 3
test_batch_size = 4

gen_lm_sample_percentage = 0.05

top_k_qa = 20
top_p_qa = 0.

FILL_VAL = -1
n_gpus = 1
device_ids = [1]

logging.basicConfig(filename=f'{MODEL_DIR}/test_run.log', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
MODEL_CLASSES = {
    'gpt2': (GPT2LMHeadModel, GPT2Tokenizer, GPT2Config),
}

In [4]:
TASK_DICT = {
    "movie": {
               "train":os.path.join(data_dir,"movie_train.json"),
               "eval":os.path.join(data_dir,"movie_dev.json"),
               "test":os.path.join(data_dir,"movie_test.json"),
               "n_train_epochs": n_train_epochs 
    },
    "boolq": {
               "train":os.path.join(data_dir,"boolq_train.json"),
               "eval":os.path.join(data_dir,"boolq_dev.json"),
               "test":os.path.join(data_dir,"boolq_test.json"),
               "n_train_epochs": n_train_epochs 
    },
    "scifact": {
               "train":os.path.join(data_dir,"scifact_train.json"),
               "eval":os.path.join(data_dir,"scifact_dev.json"),
               "test":os.path.join(data_dir,"scifact_test.json"),
               "n_train_epochs": n_train_epochs 
    }
}


In [5]:
special_tokens = {"ans_token":'__ans__', "pad_token":'__pad__', "unk_token":'__unk__', "eos_token": '<|endoftext|>'}

model_class, tokenizer_class, config_class = MODEL_CLASSES['gpt2']
tokenizer = tokenizer_class.from_pretrained('gpt2')
tokenizer.add_tokens(list(special_tokens.values()))
special_token_ids = {k:tokenizer.convert_tokens_to_ids(v) for k,v in special_tokens.items()}


model_config = config_class.from_pretrained('gpt2')
model_config.vocab_size = len(tokenizer)
max_len = model_config.n_positions

tokens_weight = torch.ones([model_config.vocab_size], dtype=torch.float).to(DEVICE)
tokens_weight[special_token_ids["ans_token"]] = 5


MODEL_CLASS = model_class
TOKENIZER = tokenizer
SPECIAL_TOKENS = special_tokens
SPECIAL_TOKEN_IDS = special_token_ids
TOKENS_WEIGHT = tokens_weight
MODEL_CONFIG = model_config

print(SPECIAL_TOKENS)
print(SPECIAL_TOKEN_IDS)

{'ans_token': '__ans__', 'pad_token': '__pad__', 'unk_token': '__unk__', 'eos_token': '<|endoftext|>'}
{'ans_token': 50257, 'pad_token': 50258, 'unk_token': 50259, 'eos_token': 50256}


In [6]:
class QADataset(Dataset):
    def __init__(self, data_paths, data_type, gen_token, extra_data=[]):
        self.data_type = data_type
        self.gen_token = gen_token
        self.ans_token = SPECIAL_TOKEN_IDS["ans_token"]
        self.eos_token = SPECIAL_TOKEN_IDS["eos_token"]
        self.pad_token = SPECIAL_TOKEN_IDS["pad_token"]

        if not isinstance(data_paths, list):
            data_paths = [data_paths]

        data = []
        for data_path in data_paths:
            if not data_path:
                continue
            with open(data_path, "r") as f:
                raw_ds = json.load(f)
            raw_ds = map(lambda x: x["paragraphs"], raw_ds["data"])
            d = []
            for raw_d in raw_ds:
                d.extend(raw_d)
            data += d
        
        self.data = []
        self.max_a_len = 0
        if len(data) > 0:
            self.data_tokenization(data)

        if len(extra_data) > 0:
            extra_data = map(lambda x: self.etl_single_extra_data(x), extra_data)
            extra_data = list(filter(lambda x:x, extra_data))
            if gen_lm_sample_percentage > 0. and len(extra_data) == 0:
                logger.warning("No good extra data but sample percentage > 0!")
            self.data += extra_data


    def etl_single_extra_data(self, data):
        gen_token = data[0]
        data = ' '.join([str(datum) for datum in data[1:]])
        try:
            context = ""
            qa = data
            question, answer = re.split(str(SPECIAL_TOKEN_IDS["ans_token"]), qa)
            context = [int(c) for c in context.strip().split()]
            question = [int(q) for q in question.strip().split()]
            answer = [int(a) for a in re.sub(str(SPECIAL_TOKEN_IDS["eos_token"]), "", answer).strip().split()]
            uid = uuid.uuid1().hex
            data = self.parse_example(gen_token, context, question, answer, uid)
        except ValueError:
            return
        return data

    def concat_example(self, gen_token, c, sep_token, q, ans_token, a, eos_token):
        example = sep_token + q + ans_token + a
        if len(example) + 1 > max_len:
            logger.warning('an example with len {} is too long!'.format(len(example) + 1))
            return
        example = gen_token + c[:max_len-len(example)-1] + example + eos_token
        return example

    def parse_example(self, gen_token, context, question, answer, idx):
        cq_example = self.concat_example([], context, [], question, [self.ans_token], [], [])
        cqa_example = self.concat_example([], context, [], question, [self.ans_token], answer, [])
        Y_example = self.concat_example([], [], [], [], [], answer, [self.eos_token])
        Y_example = [FILL_VAL] * (len(cqa_example) - len(Y_example)) + Y_example
        gen_X_example = self.concat_example([gen_token], context, [], question, [self.ans_token], answer, [])
        gen_Y_example = self.concat_example([], context, [], question, [self.ans_token], answer, [self.eos_token])
        return cq_example, len(cq_example), cqa_example, len(cqa_example), Y_example, gen_X_example, gen_Y_example, idx

    def parallel_tokenization(self, d):
        # ADD MAX LENGTH FOR MODEL SO IT DOESNT SHOW WARNING
        # OLD VERSION OF PYTORCH HUGGINGFACE DOESNT HAVE MAX LENGTH!!!!!
        # Suppress the warnings instead! https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings
        # Still doesn't work. idk what to do. we can delete all the warnings catches here
        examples = []
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            context = TOKENIZER.encode(d["context"])
        max_a_len = 0
        for qa in d["qas"]:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                question = TOKENIZER.encode(qa["question"])

            raw_answers = qa["answers"]
            if len(raw_answers) == 0:
                assert qa["is_impossible"]
                raw_answers.append({"text": ""})

            answer = []
            for i, raw_answer in enumerate(raw_answers):
                answer.extend(TOKENIZER.encode(raw_answer["text"]))
                if i != len(raw_answers) - 1:
                    answer.append(self.pad_token)
            max_a_len = max(max_a_len, len(answer))

            examples.append(self.parse_example(self.gen_token, context, question, answer, qa.get("id", 0)))
        return examples, max_a_len

    def data_tokenization(self, data):
        with Pool(4) as pool:
            data = pool.map(self.parallel_tokenization, data)
        for datum, max_a_len in data:
            self.data.extend(datum)
            self.max_a_len = max(self.max_a_len, max_a_len)

    def sort(self):
        self.data.sort(key=lambda x: len(x[0]))
        return self

    def sort_by_index(self):
        self.data.sort(key=lambda x: x[-1])

    def get_indices(self):
        return [d[-1] for d in self.data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

In [7]:
# In[9]:


class DynamicBatchSampler(Sampler):
    def __init__(self, dataset, data_type, max_batch_size):
        self.dataset = dataset
        self.data_type = data_type
        if data_type == "train":
            self.batch_size = train_batch_size
        else:
            self.batch_size = test_batch_size
        self.n_samples = len(dataset)
        self.max_batch_size = max_batch_size

    def __iter__(self):
        if self.data_type == "test":
            indices = range(self.n_samples)
        else:
            indices = np.random.permutation(self.n_samples)
        max_len, cnt, st = 0, 0, 0
        batch = []
        for ed, idx in enumerate(indices):
            ln = len(self.dataset[idx][2])
            if max(max_len, ln)**LEN_FACTOR * (ed - st + 1) > self.batch_size[cnt]:
                st = ed
                cnt += 1
                max_len = 0
                if cnt == n_gpus:
                    yield batch
                    cnt = 0
                    batch = []
            max_len = max(max_len, ln)
            batch.append(idx)
            if len(batch) == self.max_batch_size and self.data_type == "train":
                yield batch
                cnt, max_len, st = 0, 0, ed
                batch = []
        if len(batch) > 0:
            yield batch

    def __len__(self):
        raise NotImplementedError


# In[10]:


def dynamic_collate_fn(data, batch_size):

    def local_collate():
        null_counter = 0
        _cqs, _len_cqs, _cqas, _len_cqas, _Ys, _gen_Xs, _gen_Ys = [], [], [], [], [], [], []
        Y_max_len = max(len(data[j][4]) for j in range(st, ed))
        cq_max_len = max(len(data[j][0]) for j in range(st, ed))
        for j in range(st, ed):
            if None in data[j] or [] in data[j]:
                null_counter+=1
                logger.warning('null example in collate_fn, count: {}'.format(null_counter))
                continue

            pad_len = cqa_max_len - len(data[j][2])

            _cqs.append(pad_to_max_len(data[j][0], cq_max_len-len(data[j][0]), SPECIAL_TOKEN_IDS["pad_token"]))
            _len_cqs.append(data[j][1])
            _cqas.append(pad_to_max_len(data[j][2], pad_len, SPECIAL_TOKEN_IDS["pad_token"]))
            _len_cqas.append(data[j][3])
            _Ys.append(pad_to_max_len(data[j][4], Y_max_len - len(data[j][4]), FILL_VAL))
            _gen_Xs.append(pad_to_max_len(data[j][5], pad_len, SPECIAL_TOKEN_IDS["pad_token"]))
            _gen_Ys.append(pad_to_max_len(data[j][6], pad_len, FILL_VAL))

        cqs.append(torch.tensor(_cqs))
        len_cqs.append(torch.tensor(_len_cqs))
        cqas.append(torch.tensor(_cqas))
        len_cqas.append(torch.tensor(_len_cqas))
        Ys.append(torch.tensor(_Ys))
        gen_Xs.append(torch.tensor(_gen_Xs))
        gen_Ys.append(torch.tensor(_gen_Ys))

    cqs, len_cqs, cqas, len_cqas, Ys, gen_Xs, gen_Ys = [], [], [], [], [], [], []
    cqa_max_len, cnt, st = 0, 0, 0
    for ed, datum in enumerate(data):
        ln = len(datum[2]) # use cqas to calibrate
        if max(cqa_max_len, ln)**LEN_FACTOR * (ed - st + 1) > batch_size[cnt]:
            local_collate()
            cnt += 1
            cqa_max_len = 0
            st = ed
        cqa_max_len = max(cqa_max_len, ln)
    ed += 1  # otherwise ed will be len(data)-1
    local_collate()

    return cqs, len_cqs, cqas, len_cqas, Ys, gen_Xs, gen_Ys


# In[11]:



def varlen_collate_fn(data):
    batch_size = (len(data) + n_gpus - 1) // n_gpus
    cqs = torch.tensor(pad_all_to_max_len([datum[0] for datum in data], SPECIAL_TOKEN_IDS["pad_token"])).split(batch_size)
    len_cqs = torch.tensor([datum[1] for datum in data]).split(batch_size)
    cqas = torch.tensor(pad_all_to_max_len([datum[2] for datum in data], SPECIAL_TOKEN_IDS["pad_token"])).split(batch_size)
    len_cqas = torch.tensor([datum[3] for datum in data]).split(batch_size)
    Ys = torch.tensor(pad_all_to_max_len([datum[4] for datum in data], FILL_VAL)).split(batch_size)
    gen_Xs = torch.tensor(pad_all_to_max_len([datum[5] for datum in data], SPECIAL_TOKEN_IDS["pad_token"])).split(batch_size)
    gen_Ys = torch.tensor(pad_all_to_max_len([datum[6] for datum in data], FILL_VAL)).split(batch_size)
    return list(cqs), list(len_cqs), list(cqas), list(len_cqas), list(Ys), list(gen_Xs), list(gen_Ys)


# In[12]:


def pad_to_max_len(l, pad_len, val):
    return l + [val] * pad_len
def pad_all_to_max_len(ls, val):
    max_len = max(len(l) for l in ls)
    return [pad_to_max_len(l, max_len-len(l), val) for l in ls]


# In[13]:


def create_dataloader(dataset, data_type, max_batch_size=1000000000):
    if data_type == "train":
        batch_size = train_batch_size
    else:
        batch_size = test_batch_size

    if isinstance(batch_size, list):
        collate_fn=lambda x,bs=batch_size: dynamic_collate_fn(x, bs)
        shuffle = False
        batch_size = 1
        batch_sampler = DynamicBatchSampler(dataset, data_type, max_batch_size)
    else:
        collate_fn=lambda x: varlen_collate_fn(x)
#         shuffle = not (data_type != "train" or args.debug)
        shuffle = False
        batch_sampler = None

    dataloader =  DataLoader(dataset, num_workers=4,
                             collate_fn=collate_fn,
                             shuffle=shuffle,
                             batch_size=batch_size,
                             batch_sampler=batch_sampler)
    return dataloader

In [8]:
def remove_id(idx, need_process, all_pasts):
    assert idx in need_process
    del need_process[idx]
    for layer_id in range(MODEL_CONFIG.n_layer):
        all_pasts[layer_id][idx] = 0
        
        
def sample_sequence(model, need_process, qa_results, all_pasts, max_tot_lens):
    while len(need_process) > 0:                       # While there is still any need_process
        first_id = next(iter(need_process))            # The first one to process
        shortest_len = len(qa_results[first_id])       # The shortest length is the length of itself?
#         decode_batch_size = int(args.memory_sizes[0] * MEMORY_FACTOR[args.seq_train_type] // (shortest_len+1)**LEN_FACTOR)
        decode_batch_size = test_batch_size
        it = iter(need_process)                        # it is iterable of need_process
        stop = False
        remove_ids = []
        while not stop:
            batch_ids, input_ids, past = [], [], [[] for _ in range(MODEL_CONFIG.n_layer)]
            while True:
                try:
                    cur_id = next(it)                   # let the current id be the next batch of need_process
                    if len(qa_results[cur_id]) > shortest_len:  # if the length is too long, just stop
                        stop = True
                        break
                    batch_ids.append(cur_id)            
                    input_ids.append(qa_results[cur_id][-1:])
                    for layer_id in range(MODEL_CONFIG.n_layer):
                        past[layer_id].append(all_pasts[layer_id][cur_id])
                    if len(input_ids) == decode_batch_size:
                        break
                except StopIteration:                    # if there is no more id in need_process, just stop
                    stop = True
                    break

            n_inputs = len(input_ids)
            if n_inputs == 0:
                break
            input_ids = torch.stack(input_ids)
            for layer_id in range(MODEL_CONFIG.n_layer):
                past[layer_id] = torch.stack(past[layer_id], dim=1)
            all_outputs = model(input_ids=input_ids.cuda(), past=past)

            outputs = all_outputs[0]
            pasts = all_outputs[1]

            next_logits = outputs[..., -1, :] / temperature_qa
            next_tokens = logits_to_tokens(next_logits).cpu()

            for i, cur_id in enumerate(batch_ids):
                if next_tokens[i] == SPECIAL_TOKEN_IDS["eos_token"]:
                    remove_ids.append(cur_id)
                else:
                    qa_results[cur_id] = torch.cat((qa_results[cur_id], next_tokens[i]))
                    if len(qa_results[cur_id]) in [max_tot_lens[cur_id], max_len]:
                        remove_ids.append(cur_id)
                    else:
                        for layer_id in range(MODEL_CONFIG.n_layer):
                            all_pasts[layer_id][cur_id] = pasts[layer_id][:, i].type(torch.half)
        for idx in remove_ids:
            remove_id(idx, need_process, all_pasts)

def get_gen_token(task):
    return '__' + task + '__'

def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    # assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value
    return logits

def logits_to_tokens(next_logits):
    filtered_logits = top_k_top_p_filtering(next_logits, top_k=top_k_qa, top_p=top_p_qa)
    log_probs = F.softmax(filtered_logits, dim=-1)
    next_tokens = torch.multinomial(log_probs, num_samples=1)
    return next_tokens

In [9]:
def get_test_score(task_eval,qa_results,score_dict):

    score = compute_metrics(
            qa_results,
            bleu='iwslt.en.de' in task_eval or 'multinli.in.out' in task_eval,
            dialogue='woz.en' in task_eval,
            rouge='cnn_dailymail' in task_eval,
            logical_form='wikisql' in task_eval,
            corpus_f1='zre' in task_eval
    )
    score_dict[task_eval] = score


In [10]:
def test_one_to_one(task_load, task_eval, model, score_dict):
    tic_TASK = time.time()
    logger.info("start to test { task: %s (load) %s (eval)}" % (task_load, task_eval))
    print("start to test { task: %s (load) %s (eval)}" % (task_load, task_eval))

    
    # Test Dataset : Support (Train QAData) Query (Test QAData)
    support_qadata = QADataset(TASK_DICT[task_eval]["train"], "train", SPECIAL_TOKEN_IDS[task_load])
    test_qadata = QADataset(TASK_DICT[task_eval]["test"] , "test", SPECIAL_TOKEN_IDS[task_load]).sort()
    
    max_a_len = test_qadata.max_a_len
    n_examples = len(test_qadata)
    logger.info("len of test dataset: {}".format(n_examples))
    print("len of test dataset: {}".format(n_examples))
    
    ##### Make dataloaders for that particular dataset #####
    support_dataloader = create_dataloader(support_qadata, "train")
    test_dataloader = create_dataloader(test_qadata, "test")
    
    
    ##### Stream from that dataset's dataloader #####
    iter_support_dataloader = iter(support_dataloader)
    iter_test_dataloader = iter(test_dataloader)
    
    # Make model_0
    model.load_state_dict(model.state_dict)
    model_0 = model
    

    need_process = OrderedDict()
    # qa_results is qa_results[cnt]
    qa_results = [0 for _ in range(n_examples)]
    # All pasts is shape all_pasts[layer_id][cnt]
    all_pasts = [[0 for _ in range(n_examples)] for __ in range(MODEL_CONFIG.n_layer)]
    # max_tot_lens is qa_results[cnt]
    max_tot_lens = [0 for _ in range(n_examples)]

    cnt = 0
    n_steps = 0
    
    while True:
#     for n_steps, (cq, len_cq, cqa, len_cqa, Y, genX, genY) in enumerate(test_dataloader):
        # 1. Get the support data from the train dataloader
        #    and the query data from the test dataloader
        # Assume that query data >> support data!  - cant do this. try twice.
        try:
            _, _, support_x, _, support_y, _, _ = next(iter_support_dataloader)
        except StopIteration:
            iter_support_dataloader = iter(support_dataloader)
            _, _, support_x, _, support_y, _, _ = next(iter_support_dataloader)
        try:
            query_x, query_x_len, query_x_cqa, _, query_y, _, _ = next(iter_test_dataloader) # Let query get the CQ!
        except StopIteration:
            break
            
        
        # Different inputs for train and test -> train with batch 3 and test with batch 1
        n_inputs_train = sum(_cqa.shape[0] for _cqa in support_x)
        n_inputs = sum(_cqa.shape[0] for _cqa in query_x)

        # Since we only have 1 GPU, just use the first one, it will separate batches according to the device IDS
        support_x = support_x[0]
        support_y = support_y[0]
        query_x = query_x[0]
        query_y = query_y[0]
        query_x_len = query_x_len[0] # an array of query x lengths, but test batch size is only1??
        query_x_cqa = query_x_cqa[0] #EXTRA DEBUG

        support_x = support_x.to(DEVICE)
        support_y = support_y.to(DEVICE)
        query_x = query_x.to(DEVICE)
        query_y = query_y
        query_x_cqa = query_x_cqa.to(DEVICE) #EXTRA DEBUG
        
        
        ### START Adaptation Phase ###
        # 2. Reinitialize model with parameters from model_path
        model = copy.deepcopy(model_0)
        model.train()
        
        # Training loss function
        train_loss_fct = CrossEntropyLoss(ignore_index=FILL_VAL, weight=TOKENS_WEIGHT)
        
        # Optimizer
        max_grad_norm=1
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

        optimizer = AdamW(optimizer_grouped_parameters, lr=6.25e-5, eps=1e-4)
        optimizer = FP16_Optimizer(optimizer, static_loss_scale=None, dynamic_loss_scale=True,
                                           dynamic_loss_args={'scale_window': 100, 'min_scale': 1, 'delayed_shift': 2})

        
        
        # 3. Update the weights with the support set
        # May update for several steps
        for i in range(num_updates):

            qa_logits = model(support_x)
            # Somehow it also returns attentions in [1]?, this is selecting 0 of what WrapModel is doing 
            qa_logits = qa_logits[0]
            qa_loss = train_loss_fct(qa_logits.transpose(1,2), support_y)
            loss = qa_loss

            logger.info(f"[DEBUG] Adaptation loss: {qa_loss.item()}")
            # Update Optimizer
            optimizer.backward(loss, update_master_grads=False) # instead of loss.backward() for fp16
            optimizer.update_master_grads()
            optimizer.clip_master_grads(max_grad_norm)
            optimizer.step()
            # Ignore this for now
#             if not optimizer.overflow:
#                 for i in range(n_inputs):
#                     scheduler.step()
            optimizer.zero_grad()
        ### END Adaptation Phase ###
        
        model.eval()

        ### START Meta-Learning Phase ###
        # 4. After Adaptation, use the query set for test (CQ ONLY)
        # model() returns Tuple of length 2: 
        #  The [0] is a  torch.Size([1, 225, 50260]), and the [1] is 12 of torch.Size([2, 1, 12, 225, 64])
        # Thinking that the [0] is the actual output and [1] is the pasts?
        all_outputs = model(query_x)
        outputs = all_outputs[0]
        pasts = all_outputs[1]
        next_logits = outputs[range(n_inputs), query_x_len-1, :] / temperature_qa
        next_tokens = logits_to_tokens(next_logits).cpu()
        
        
        # EXTRA FOR COMPARE
        qa_logits = model(query_x_cqa)[0]
        qa_loss = train_loss_fct(qa_logits.transpose(1,2), query_y.to(DEVICE))
        logger.info(f"[DEBUG] QUERY LOSS: {qa_loss.item()}")
        
        # Maybe this is not needed in testing since n_inputs is only 1??
        for batch_i in range(n_inputs):
            # max total length = max answer length + length of cq
            max_tot_lens[cnt] = max_a_len + test_qadata[cnt][1] 
            # add the cq of that particular batch to qa_results (Change it to cpu first!)
            qa_results[cnt] = query_x.cpu()[batch_i][:query_x_len[batch_i]]
            
            # If the next tokens is not eos
            if next_tokens[batch_i] != SPECIAL_TOKEN_IDS["eos_token"]:
                # Concat the result
                qa_results[cnt] = torch.cat((qa_results[cnt], next_tokens[batch_i]))
                # if the length is not max yet -> MAXTOT 225 1024
                if len(qa_results[cnt]) not in [max_tot_lens[cnt], max_len]:
                    # Append need_process of that cnt
                    need_process.update([[cnt, None]])
                    # Update all pasts
                    for layer_id in range(MODEL_CONFIG.n_layer):
                        all_pasts[layer_id][cnt] = pasts[layer_id][:, batch_i, ..., :query_x_len[batch_i], :].type(torch.half)
            
            # Try sample_sequence here! it will get all need_process (should be only 1 batch, and generate all!)
            sample_sequence(model, need_process, qa_results, all_pasts, max_tot_lens)
            
            logger.info(f"[ERROR_ANALYSIS] {task_eval} {cnt}/{n_examples} Predicted Answer {TOKENIZER.decode(qa_results[cnt].tolist())}")
            logger.info(f"[ERROR_ANALYSIS] {task_eval} {cnt}/{n_examples} Predicted Tokens {qa_results[cnt].tolist()[query_x_len[batch_i]:]}")

            # Do the score calculation here
            # The answer of that particular batch to list
            Y = query_y[batch_i].tolist()
            Y = list(filter(lambda x: x != -1, Y))[:-1]  # remove eos from the answer
            logger.info(f"[ERROR_ANALYSIS] {task_eval} {cnt}/{n_examples} Actual Tokens {Y}")
            Y = ' '.join([str(y) for y in Y]).split(str(SPECIAL_TOKEN_IDS["pad_token"]))
            Y = [TOKENIZER.decode(list(map(int, y.split()))) for y in Y]
            # Change the QA Results to a decoded version of real answer and predicted answer
            qa_results[cnt] = [TOKENIZER.decode(qa_results[cnt].tolist()[query_x_len[batch_i]:]), Y]
            print(f"Predict vs Actual {cnt}/{n_examples}", qa_results[cnt])
            logger.info(f"[ERROR_ANALYSIS] {task_eval} {cnt}/{n_examples} Actual Answer {Y}")
            logger.info(f"[ERROR_ANALYSIS] {task_eval} {cnt}/{n_examples} Predict vs Actual {qa_results[cnt]}")
            
            cnt += 1
        n_steps += 1
        
    toc_TASK = time.time() - tic_TASK
    logger.info(f'[TIME] TASK {(task_load, task_eval)} {toc_TASK}')
    
    get_test_score(task_eval, qa_results, score_dict)
    print(score_dict)

    model_dir = model.model_dir
    results_path = os.path.join(model_dir,f"qa_{task_eval}.csv")
    with open(results_path, "w",encoding="utf-8") as f:
        qa_writer = csv.writer(f,delimiter=',')
        qa_writer.writerow(["y","pred"])
        for pred, y in qa_results:
            qa_writer.writerow([y,pred]) 

    return model, score_dict

In [11]:
%%time

for task in tasks:
    
    model_path = os.path.join(MODEL_DIR, f"{task}.model")
    config_path = os.path.join(MODEL_DIR,CONFIG_NAME)

    gen_token = get_gen_token(task)
    TOKENIZER.add_tokens([gen_token])
    SPECIAL_TOKENS[task] = gen_token
    SPECIAL_TOKEN_IDS[task] = TOKENIZER.convert_tokens_to_ids(gen_token)
#     model_config = CONFIG_CLASS.from_json_file(config_path) # Already defined
    model = MODEL_CLASS(model_config).cuda()
    # Don't load state dict here, load for every adaptation phase!
    
#     print(model)
    print(model_path)
    
    global TOKENS_WEIGHT
    if len(TOKENIZER) != TOKENS_WEIGHT.shape[0]:
        TOKENS_WEIGHT = torch.cat((TOKENS_WEIGHT, torch.ones([1]).cuda()))
    
    model.resize_token_embeddings(len(TOKENIZER))
    model = FP16_Module(model)
    
    
    model.model_dir = MODEL_DIR
    model.model_path = model_path
    # Try Loading the state dict like this!
    model.state_dict = torch.load(model.model_path, map_location='cuda:0')
    logger.info(f"task: {task}")
    score_dict = {k:None for k in tasks}
    
    for task_eval in tasks:
        test_one_to_one(task, task_eval, model, score_dict)
    logger.info("score: {}".format(score_dict))

    with open(os.path.join(MODEL_DIR, f"metrics-{task}.json"),"w") as f:
        json.dump(score_dict, f)

/data/model_runs/20210905T094223_sbm_SEQ_MAML/scifact.model
start to test { task: scifact (load) scifact (eval)}
len of test dataset: 188
Predict vs Actual 0/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 1/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 2/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 3/188 [' CONCLUSIONS', [' SUPPORTS']]
Predict vs Actual 4/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 5/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 6/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 7/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 8/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 9/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 10/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 11/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 12/188 ['', [' SUPPORTS']]
Predict vs Actual 13/188 [' F', [' SUPPORTS']]
Predict vs Actual 14/188 ['\n __unk__ SUPP', [' SUPPORTS']]
Predict vs Actual 15/188 ['', [' REFUTES']]
Predict vs Actual 16/188 [' __

Predict vs Actual 156/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 157/188 [' REFUTES', [' REFUTES']]
Predict vs Actual 158/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 159/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 160/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 161/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 162/188 [' REFUTES', [' SUPPORTS']]
Predict vs Actual 163/188 [' REFUTES', [' SUPPORTS']]
Predict vs Actual 164/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 165/188 [' REFUTES', [' SUPPORTS']]
Predict vs Actual 166/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 167/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 168/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 169/188 [' REFUTES', [' SUPPORTS']]
Predict vs Actual 170/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 171/188 [' REFUTES', [' SUPPORTS']]
Predict vs Actual 172/188 [' SUPPORTS', [' REFUTES']]
Predict vs Actual 173/188 [' SUPPORTS', [' SUPPORTS']]
Predict vs Actual 174/

Predict vs Actual 136/2807 [' PER', [' True']]
Predict vs Actual 137/2807 [' RE', [' True']]
Predict vs Actual 138/2807 [' SUP', [' True']]
Predict vs Actual 139/2807 [' RE', [' True']]
Predict vs Actual 140/2807 [' __unk__', [' True']]
Predict vs Actual 141/2807 ['', [' True']]
Predict vs Actual 142/2807 [' RE', [' False']]
Predict vs Actual 143/2807 [' SUP', [' True']]
Predict vs Actual 144/2807 [' SUP', [' False']]
Predict vs Actual 145/2807 [' SUP', [' False']]
Predict vs Actual 146/2807 [' RE', [' False']]
Predict vs Actual 147/2807 [' SUP', [' True']]
Predict vs Actual 148/2807 [' SUP', [' False']]
Predict vs Actual 149/2807 [' RE', [' False']]
Predict vs Actual 150/2807 [' __pad__', [' True']]
Predict vs Actual 151/2807 [' SUP', [' False']]
Predict vs Actual 152/2807 [' __ans__', [' False']]
Predict vs Actual 153/2807 [' __unk__', [' False']]
Predict vs Actual 154/2807 [' SUP', [' False']]
Predict vs Actual 155/2807 [' SUP', [' True']]
Predict vs Actual 156/2807 [' __unk__', [' 

Predict vs Actual 308/2807 [' LEG', [' False']]
Predict vs Actual 309/2807 [' SUP', [' True']]
Predict vs Actual 310/2807 [' CON', [' True']]
Predict vs Actual 311/2807 [' S', [' True']]
Predict vs Actual 312/2807 ['', [' True']]
Predict vs Actual 313/2807 [' __unk__', [' True']]
Predict vs Actual 314/2807 [' __scifact__', [' True']]
Predict vs Actual 315/2807 [' __unk__', [' True']]
Predict vs Actual 316/2807 [' SUP', [' False']]
Predict vs Actual 317/2807 [' __unk__', [' False']]
Predict vs Actual 318/2807 [' __unk__', [' True']]
Predict vs Actual 319/2807 [' __unk__', [' False']]
Predict vs Actual 320/2807 [' SUP', [' False']]
Predict vs Actual 321/2807 [' __pad__', [' False']]
Predict vs Actual 322/2807 ['', [' True']]
Predict vs Actual 323/2807 [' __ans__', [' True']]
Predict vs Actual 324/2807 [' SUP', [' False']]
Predict vs Actual 325/2807 [' CONT', [' False']]
Predict vs Actual 326/2807 [' OFF', [' True']]
Predict vs Actual 327/2807 ['', [' False']]
Predict vs Actual 328/2807 [

Predict vs Actual 488/2807 [' __unk__', [' False']]
Predict vs Actual 489/2807 [' SUP', [' True']]
Predict vs Actual 490/2807 [' __ans__', [' False']]
Predict vs Actual 491/2807 ['', [' False']]
Predict vs Actual 492/2807 [' COMP', [' True']]
Predict vs Actual 493/2807 ['', [' True']]
Predict vs Actual 494/2807 [' __ans__', [' True']]
Predict vs Actual 495/2807 ['', [' True']]
Predict vs Actual 496/2807 [' IN', [' False']]
Predict vs Actual 497/2807 ['', [' True']]
Predict vs Actual 498/2807 ['', [' False']]
Predict vs Actual 499/2807 [' SUP', [' False']]
Predict vs Actual 500/2807 [' SUP', [' False']]
Predict vs Actual 501/2807 [' SUP', [' False']]
Predict vs Actual 502/2807 [' SUP', [' True']]
Predict vs Actual 503/2807 [' SUP', [' False']]
Predict vs Actual 504/2807 ['', [' True']]
Predict vs Actual 505/2807 [' SUP', [' True']]
Predict vs Actual 506/2807 [' F', [' False']]
Predict vs Actual 507/2807 [' __unk__', [' True']]
Predict vs Actual 508/2807 ['', [' False']]
Predict vs Actua

...

Predict vs Actual 2208/2807 [' POS', [' True']]
Predict vs Actual 2209/2807 [' POS', [' True']]
Predict vs Actual 2210/2807 [' POS', [' True']]
Predict vs Actual 2211/2807 [' POS', [' False']]
Predict vs Actual 2212/2807 [' POS', [' True']]
Predict vs Actual 2213/2807 [' POS', [' False']]
Predict vs Actual 2214/2807 [' POS', [' False']]
Predict vs Actual 2215/2807 [' POS', [' True']]
Predict vs Actual 2216/2807 [' POS', [' False']]
Predict vs Actual 2217/2807 [' POS', [' False']]
Predict vs Actual 2218/2807 [' POS', [' False']]
Predict vs Actual 2219/2807 [' POS', [' False']]
Predict vs Actual 2220/2807 [' POS', [' True']]
Predict vs Actual 2221/2807 [' POS', [' False']]
Predict vs Actual 2222/2807 [' POS', [' False']]
Predict vs Actual 2223/2807 [' POS', [' False']]
Predict vs Actual 2224/2807 [' POS', [' False']]
Predict vs Actual 2225/2807 [' POS', [' True']]
Predict vs Actual 2226/2807 [' POS', [' False']]
Predict vs Actual 2227/2807 [' POS', [' True']]
Predict vs Actual 2228/2807 

Predict vs Actual 2380/2807 [' POS', [' True']]
Predict vs Actual 2381/2807 [' POS', [' False']]
Predict vs Actual 2382/2807 [' POS', [' True']]
Predict vs Actual 2383/2807 [' POS', [' True']]
Predict vs Actual 2384/2807 [' POS', [' True']]
Predict vs Actual 2385/2807 [' POS', [' False']]
Predict vs Actual 2386/2807 [' POS', [' False']]
Predict vs Actual 2387/2807 [' POS', [' False']]
Predict vs Actual 2388/2807 [' POS', [' True']]
Predict vs Actual 2389/2807 [' POS', [' False']]
Predict vs Actual 2390/2807 [' POS', [' True']]
Predict vs Actual 2391/2807 [' POS', [' False']]
Predict vs Actual 2392/2807 [' POS', [' False']]
Predict vs Actual 2393/2807 [' POS', [' True']]
Predict vs Actual 2394/2807 [' POS', [' True']]
Predict vs Actual 2395/2807 [' POS', [' True']]
Predict vs Actual 2396/2807 [' POS', [' True']]
Predict vs Actual 2397/2807 [' POS', [' True']]
Predict vs Actual 2398/2807 [' POS', [' False']]
Predict vs Actual 2399/2807 [' POS', [' False']]
Predict vs Actual 2400/2807 [' 

Predict vs Actual 2552/2807 [' POS', [' False']]
Predict vs Actual 2553/2807 [' POS', [' False']]
Predict vs Actual 2554/2807 [' POS', [' False']]
Predict vs Actual 2555/2807 [' POS', [' True']]
Predict vs Actual 2556/2807 [' POS', [' True']]
Predict vs Actual 2557/2807 [' POS', [' True']]
Predict vs Actual 2558/2807 [' POS', [' False']]
Predict vs Actual 2559/2807 [' POS', [' True']]
Predict vs Actual 2560/2807 [' POS', [' False']]
Predict vs Actual 2561/2807 [' POS', [' True']]
Predict vs Actual 2562/2807 [' POS', [' True']]
Predict vs Actual 2563/2807 [' POS', [' True']]
Predict vs Actual 2564/2807 [' POS', [' True']]
Predict vs Actual 2565/2807 [' POS', [' True']]
Predict vs Actual 2566/2807 [' POS', [' True']]
Predict vs Actual 2567/2807 [' POS', [' False']]
Predict vs Actual 2568/2807 [' POS', [' False']]
Predict vs Actual 2569/2807 [' POS', [' False']]
Predict vs Actual 2570/2807 [' POS', [' True']]
Predict vs Actual 2571/2807 [' POS', [' True']]
Predict vs Actual 2572/2807 [' P

Predict vs Actual 2724/2807 [' POS', [' False']]
Predict vs Actual 2725/2807 [' POS', [' False']]
Predict vs Actual 2726/2807 [' POS', [' True']]
Predict vs Actual 2727/2807 [' POS', [' False']]
Predict vs Actual 2728/2807 [' POS', [' True']]
Predict vs Actual 2729/2807 [' POS', [' False']]
Predict vs Actual 2730/2807 [' POS', [' True']]
Predict vs Actual 2731/2807 [' POS', [' True']]
Predict vs Actual 2732/2807 [' POS', [' True']]
Predict vs Actual 2733/2807 [' POS', [' True']]
Predict vs Actual 2734/2807 [' POS', [' False']]
Predict vs Actual 2735/2807 [' POS', [' True']]
Predict vs Actual 2736/2807 [' POS', [' False']]
Predict vs Actual 2737/2807 [' POS', [' False']]
Predict vs Actual 2738/2807 [' POS', [' False']]
Predict vs Actual 2739/2807 [' POS', [' True']]
Predict vs Actual 2740/2807 [' POS', [' True']]
Predict vs Actual 2741/2807 [' POS', [' True']]
Predict vs Actual 2742/2807 [' POS', [' True']]
Predict vs Actual 2743/2807 [' POS', [' True']]
Predict vs Actual 2744/2807 [' P

Predict vs Actual 92/200 [' POS', [' NEG']]
Predict vs Actual 93/200 [' POS', [' POS']]
Predict vs Actual 94/200 [' POS', [' NEG']]
Predict vs Actual 95/200 [' POS', [' NEG']]
Predict vs Actual 96/200 [' POS', [' NEG']]
Predict vs Actual 97/200 [' POS', [' POS']]
Predict vs Actual 98/200 [' POS', [' POS']]
Predict vs Actual 99/200 [' POS', [' POS']]
Predict vs Actual 100/200 [' POS', [' POS']]
Predict vs Actual 101/200 [' POS', [' POS']]
Predict vs Actual 102/200 [' POS', [' NEG']]
Predict vs Actual 103/200 [' POS', [' NEG']]
Predict vs Actual 104/200 [' POS', [' POS']]
Predict vs Actual 105/200 [' POS', [' NEG']]
Predict vs Actual 106/200 [' POS', [' POS']]
Predict vs Actual 107/200 [' POS', [' NEG']]
Predict vs Actual 108/200 [' POS', [' POS']]
Predict vs Actual 109/200 [' POS', [' NEG']]
Predict vs Actual 110/200 [' POS', [' NEG']]
Predict vs Actual 111/200 [' POS', [' NEG']]
Predict vs Actual 112/200 [' POS', [' POS']]
Predict vs Actual 113/200 [' POS', [' POS']]
Predict vs Actual 